In [2]:
import pathlib
import random 

import numpy as np 
import pandas as pd
import geopandas as gpd
import folium
import shapely 

### 0. 교통량 데이터 gid 붙이기

In [19]:
gid = pd.read_csv('C:/Users/Kim Won Gyeung/Desktop/공모전/2021 빅데이터 분석 및 아이디어 공모전/2.사용_데이터(정제데이터)/1.QGIS에 사용할 데이터(gid 가능데이터)/gid포함 csv파일/gid_210523_02.csv',encoding='cp949')
traffic = pd.read_csv('C:/Users/Kim Won Gyeung/Desktop/공모전/2021 빅데이터 분석 및 아이디어 공모전/2.사용_데이터(정제데이터)/1.QGIS에 사용할 데이터(gid 가능데이터)/gid포함 csv파일/교통량gid.csv',encoding='cp949')

In [20]:
len(traffic)

35764

#### traffic 값이 중복되는 교통량이 있어서 격자의 교통량 중 제일 높은 교통량 현황을 제시

In [21]:
traffic=traffic.groupby('gid').max()
len(traffic)

12294

In [22]:
gid = pd.merge(gid,traffic,on='gid',how='outer')
gid

,gid,차량등록수,생산가능인구수,교통노드,건물연면적,관광상권,부동산상권,생활서비스상권,소매상권,숙박상권,스포츠상권,음식상권,학문상권,유치원초중고,화재발생건수,congestion
0,다마846996,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
1,다마847995,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
2,다마847996,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
3,다마847997,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
4,다마847998,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,라바052223,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
54908,라바052224,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
54909,라바053222,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN
54910,라바053223,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN


In [23]:
gid = gid.fillna(0)

In [24]:
gid

,gid,차량등록수,생산가능인구수,교통노드,건물연면적,관광상권,부동산상권,생활서비스상권,소매상권,숙박상권,스포츠상권,음식상권,학문상권,유치원초중고,화재발생건수,congestion
0,다마846996,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
1,다마847995,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
2,다마847996,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
3,다마847997,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
4,다마847998,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,라바052223,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
54908,라바052224,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
54909,라바053222,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0
54910,라바053223,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0


### 1. 데이터 불러오기

In [39]:
h_km_2 = pd.read_csv('C:/Users/Kim Won Gyeung/Desktop/공모전/2021 빅데이터 분석 및 아이디어 공모전/2.사용_데이터(정제데이터)/1.QGIS에 사용할 데이터(gid 가능데이터)/gid포함 csv파일/수소차버퍼, lpg버퍼 격자데이터/수소충전소버퍼2km.csv', sep=",", encoding='cp949') 
h_km_2['종속변수_수소차']=1
h_km_2 = h_km_2.iloc[:,4:]

h_km_1_5 = pd.read_csv('C:/Users/Kim Won Gyeung/Desktop/공모전/2021 빅데이터 분석 및 아이디어 공모전/2.사용_데이터(정제데이터)/1.QGIS에 사용할 데이터(gid 가능데이터)/gid포함 csv파일/수소차버퍼, lpg버퍼 격자데이터/수소충전소버퍼1.5km.csv', sep=",", encoding='cp949') 
h_km_1_5['종속변수_수소차']=1
h_km_1_5 = h_km_1_5.iloc[:,4:]

h_km_1 = pd.read_csv('C:/Users/Kim Won Gyeung/Desktop/공모전/2021 빅데이터 분석 및 아이디어 공모전/2.사용_데이터(정제데이터)/1.QGIS에 사용할 데이터(gid 가능데이터)/gid포함 csv파일/수소차버퍼, lpg버퍼 격자데이터/수소충전소버퍼1km.csv', sep=",", encoding='cp949') 
h_km_1['종속변수_수소차']=1
h_km_1 = h_km_1.iloc[:,4:]

h_km_0_5 = pd.read_csv('C:/Users/Kim Won Gyeung/Desktop/공모전/2021 빅데이터 분석 및 아이디어 공모전/2.사용_데이터(정제데이터)/1.QGIS에 사용할 데이터(gid 가능데이터)/gid포함 csv파일/수소차버퍼, lpg버퍼 격자데이터/수소충전소버퍼0.5km.csv', sep=",", encoding='cp949') 
h_km_0_5['종속변수_수소차']=1
h_km_0_5 = h_km_0_5.iloc[:,4:]


print(len(h_km_2),len(h_km_1_5),len(h_km_1),len(h_km_0_5))

7374 4320 2027 577


In [41]:
h_km_2

,gid,종속변수_수소차
0,다바823157,1
1,다바821164,1
2,다바844151,1
3,다바831152,1
4,다바830145,1
...,...,...
7369,다바907300,1
7370,다바898275,1
7371,다바889286,1
7372,다바900286,1


### 2. 중복 데이터 제거

In [42]:
h_km_2=h_km_2.groupby('gid').max()
h_km_2.reset_index(inplace=True)

h_km_1_5=h_km_1_5.groupby('gid').max()
h_km_1_5.reset_index(inplace=True)

h_km_1=h_km_1.groupby('gid').max()
h_km_1.reset_index(inplace=True)

h_km_0_5=h_km_0_5.groupby('gid').max()
h_km_0_5.reset_index(inplace=True)

print(len(h_km_2),len(h_km_1_5),len(h_km_1),len(h_km_0_5))

6990 4250 2027 577


### 3. km별 데이터 합치기

In [46]:
### 3-1. 수소차 2km 반경
gid_2km = pd.merge(gid,h_km_2,on='gid',how='outer')
gid_2km = gid_2km.fillna(0)

In [49]:
### 3-1. 수소차 1.5km 반경
gid_1_5km = pd.merge(gid,h_km_1_5,on='gid',how='outer')
gid_1_5km = gid_1_5km.fillna(0)

In [50]:
### 3-1. 수소차 1km 반경
gid_1km = pd.merge(gid,h_km_1,on='gid',how='outer')
gid_1km = gid_1km.fillna(0)

In [51]:
### 3-1. 수소차 0.5km 반경
gid_0_5km = pd.merge(gid,h_km_0_5,on='gid',how='outer')
gid_0_5km = gid_0_5km.fillna(0)

In [52]:
gid_2km.to_csv('gid_2km.csv',index=False, encoding='cp949')
gid_1_5km.to_csv('gid_1_5km.csv',index=False, encoding='cp949')
gid_1km.to_csv('gid_1km.csv',index=False, encoding='cp949')
gid_0_5km.to_csv('gid_0_5km.csv',index=False, encoding='cp949')